# препроцессинг

In [ ]:
!git clone https://github.com/yualapshina/distrib-lab2.git --quiet

In [ ]:
import os
import random

# основной пайплайн

In [ ]:
# формат ввода:
# целое число - размер
# матрица вещественных чисел размер х размер
# вектор вещественных чисел длиной размер

# редактировать ввод
size = 3

input_content = str(size) + "\n";
for i in range(size + 1):
  for j in range(size):
    input_content += str(random.randint(0, 50)) + " "
  input_content += "\n"

with open("distrib-lab2/input_vector.txt", "w") as fin:
  fin.write(input_content);

In [ ]:
# проверить ввод
with open("distrib-lab2/input_vector.txt", "r") as fin:
  check_content = fin.read()
print(check_content)

3
27 29 41 
30 29 45 
4 10 46 
28 4 45 



In [ ]:
# количество процессов
os.environ['p'] = '5'

In [ ]:
# чё по страшному коду
# бенчмарк - однопоточный для сравнения времени и математической корректности
# (если они вдруг начнут неправильно считать, я ничего не буду делать))))))
# и две проги, третья с блоками неправильная по алгоритму, но будет приложена как свидетельство старания

In [ ]:
%%shell
cd distrib-lab2

mpicc benchmark_vector.c -o benchmark_vector -lm
mpiexec --allow-run-as-root ./benchmark_vector input_vector.txt

mpicc mpi_rows.c -o mpi_rows -lm
mpiexec --oversubscribe -np $p --allow-run-as-root ./mpi_rows input_vector.txt

mpicc mpi_columns.c -o mpi_columns -lm
mpiexec --oversubscribe -np $p --allow-run-as-root ./mpi_columns input_vector.txt

Benchmark
Time: 0.000002 seconds
Result: 2717.00 2981.00 2222.00 

MPI Rows
Time: 0.004091 seconds
Result: 2717.00 2981.00 2222.00 

MPI Columns
Time: 0.003823 seconds
Result: 2717.00 2981.00 2222.00 



# оценка

In [ ]:
# чуть более подробный отчёт с описанием алгоритмов я напишу, но попозже

# что делать с картинками:
# в прошлый раз у нас, можно сказать, был набор из трёх картинок: время, ускорение и эффективность,
# на каждой из которых цветами различалось количество потоков
# нужно, видимо, сделать два таких набора, для рядов и для столбцов, каждую программу поварьировать с большими данными и с процессами
# хорошо хоть, что запускаются вместе теперь
# однопоточную можно тоже воткнуть на график ради примера или если будет не интересно, можно не трогать